In [1]:
%cd ../src
%pwd

/home/muchio1124/Work/Study/NES_detection/src


'/home/muchio1124/Work/Study/NES_detection/src'

In [2]:
import json

virus = 'HIV-1'

with open('../references/motif_data.json', 'rb') as f:
    motif_data = json.load(f)

motif_dict = [motif_dict for motif_dict in motif_data if motif_dict['virus'] == virus]
motif_dict = motif_dict[0]
proteins = motif_dict['proteins']

In [3]:
from dataset import Dataset

length = 26
separate_len=1

dataset = Dataset(motif_dict['SLiM'],
                  idx=motif_dict['start_index'],
                  length=length,
                  proteins=proteins,
                  SLiM_protein=motif_dict['SLiM_protein'],
                  neighbor=motif_dict['neighbor'],
                  replacement_tolerance=motif_dict['replacement_tolerance'],
                  threshold=len(motif_dict['SLiM']),
                  separate_len=separate_len)

In [4]:
from Bio import SeqIO
from utils import determine_protein_name

filename_in = "../data/interim/HIV-1.fasta"
filename_out = "../data/interim/HIV-1_new.fasta"

f_in = open(filename_in, 'r')
f_out = open(filename_out, 'w')

for record in SeqIO.parse(f_in, 'fasta'):
    protein_name = determine_protein_name(record.description, proteins=proteins)

    if protein_name == motif_dict['SLiM_protein']:
        seqs, labels = dataset.make_dataset([record], dict=False)
        
        if labels.sum() == 0:
            continue
        
    SeqIO.write(record, f_out, 'fasta')
    
f_in.close()
f_out.close()